<a href="https://colab.research.google.com/github/MsrOhkwr/gpu_path_tracer_with_google_colaboratory/blob/api_test/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install pyngrok

In [3]:
%%file api.py

from flask import Flask, Response

from engine import Renderer

app = Flask(__name__)

renderer = Renderer()

@app.route("/")
def index():
    response = Response(renderer.render())
    response.headers["Access-Control-Allow-Origin"] = "*"
    response.headers["Content-type"] = "multipart/x-mixed-replace; boundary=frame"
    return response

Overwriting api.py


In [4]:
%%file engine.py

import requests

class Renderer:
    def __init__(self):
        url = "https://raw.githubusercontent.com/opencv/opencv/master/samples/data/butterfly.jpg"

        with requests.get(url) as response:
            self.content = response.content


    def render(self):
        while True:
            yield b"--frame\r\nContent-Type: image/jpeg\r\n\r\n" + self.content + b"\r\n"

Overwriting engine.py


In [0]:
from subprocess import Popen
import psutil

try:
    for child in psutil.Process(p.pid).children(recursive=True):
        child.kill()
    p.kill()
except:
    pass

cmd = ["gunicorn", "api:app", "-b", "0.0.0.0:80", "-k", "gevent"]
p = Popen(cmd)

In [6]:
from pyngrok import ngrok
import socket

host = socket.gethostbyname(socket.getfqdn(socket.gethostname()))

try:
    tunnels = ngrok.get_tunnels()
    for tunnel in tunnels:
        ngrok.disconnect(tunnel.public_url)
except:
    pass

http_url = ngrok.connect(name=host)
https_url = http_url.replace("http", "https")
print(https_url)

https://9864c6d9.ngrok.io


In [7]:
%%file index.html

<img src="$https_url">

Overwriting index.html


In [8]:
from IPython.display import HTML
from string import Template

src = Template(open("index.html").read()).substitute(https_url=https_url)

HTML(src)